<div class="alert alert-block alert-info">

## SQL   

Коронавирус застал мир врасплох, изменив привычный порядок вещей. В свободное время жители городов больше не выходят на улицу, не посещают кафе и торговые центры. Зато стало больше времени для книг. Это заметили стартаперы — и бросились создавать приложения для тех, кто любит читать.

Ваша компания решила быть на волне и купила крупный сервис для чтения книг по подписке. Ваша первая задача как аналитика — проанализировать базу данных.
В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.  
    
</div>

### Цели исследования:
- определить объем вводных данных(количество актуальной литературы);
- для каждой книги посчитать количество обзоров и среднюю оценку;
- определить издательство с максимальным выпуском;
- определить самого популярного автора;
- посчитать количество обзоров на литературу.

Импортируем необходимые библиотеки.

In [1]:
import pandas as pd
from sqlalchemy import create_engine 

In [2]:
#pip install psycopg2

Произведем подключение к базе данных.

In [3]:
db_config = {'user': 'praktikum_student', # имя пользователя
             'pwd': 'Sdf4$2;d-d30pp', # пароль
             'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
             'port': 6432, # порт подключения
             'db': 'data-analyst-final-project-db'} # название базы данных

In [4]:
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                         db_config['pwd'],
                                                         db_config['host'],
                                                         db_config['port'],
                                                         db_config['db']) 

In [5]:
engine = create_engine(connection_string, connect_args={'sslmode':'require'}) 

Создадим функцию для просмотра таблиц.

In [6]:
def show_table(table_name):
    query = """SELECT * FROM {} LIMIT 5""".format(table_name)
    return pd.io.sql.read_sql(query, con = engine)

А также функцию для просмотра общей информации о таблицах.

In [7]:
def info_table(table_name):
    query = """SELECT * FROM {}""".format(table_name)
    return pd.io.sql.read_sql(query, con = engine).info()

### Описание данных

**Таблица `books`**

Содержит данные о книгах:

- `book_id` — идентификатор книги;
- `author_id` — идентификатор автора;
- `title` — название книги;
- `num_pages` — количество страниц;
- `publication_date` — дата публикации книги;
- `publisher_id` — идентификатор издателя.

**Таблица `authors`**

Содержит данные об авторах:

- `author_id` — идентификатор автора;
- `author` — имя автора.

**Таблица `publishers`**

Содержит данные об издательствах:

- `publisher_id` — идентификатор издательства;
- `publisher` — название издательства;

**Таблица `ratings`**

Содержит данные о пользовательских оценках книг:

- `rating_id` — идентификатор оценки;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, оставившего оценку;
- `rating` — оценка книги.

**Таблица `reviews`**

Содержит данные о пользовательских обзорах на книги:

- `review_id` — идентификатор обзора;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, написавшего обзор;
- `text` — текст обзора.

Исследуем данные нам таблицы. Выведем первые 5 строк.

И общую информацию о таблицах.

In [8]:
show_table('books')

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [9]:
info_table('books')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
book_id             1000 non-null int64
author_id           1000 non-null int64
title               1000 non-null object
num_pages           1000 non-null int64
publication_date    1000 non-null object
publisher_id        1000 non-null int64
dtypes: int64(4), object(2)
memory usage: 47.0+ KB


In [10]:
show_table('authors')

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [11]:
info_table('authors')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 636 entries, 0 to 635
Data columns (total 2 columns):
author_id    636 non-null int64
author       636 non-null object
dtypes: int64(1), object(1)
memory usage: 10.1+ KB


In [12]:
show_table('publishers')

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [13]:
info_table('publishers')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 340 entries, 0 to 339
Data columns (total 2 columns):
publisher_id    340 non-null int64
publisher       340 non-null object
dtypes: int64(1), object(1)
memory usage: 5.4+ KB


In [14]:
show_table('ratings')

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [15]:
info_table('ratings')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6456 entries, 0 to 6455
Data columns (total 4 columns):
rating_id    6456 non-null int64
book_id      6456 non-null int64
username     6456 non-null object
rating       6456 non-null int64
dtypes: int64(3), object(1)
memory usage: 201.9+ KB


In [16]:
show_table('reviews')

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


In [17]:
info_table('reviews')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2793 entries, 0 to 2792
Data columns (total 4 columns):
review_id    2793 non-null int64
book_id      2793 non-null int64
username     2793 non-null object
text         2793 non-null object
dtypes: int64(2), object(2)
memory usage: 87.4+ KB


Приступим к выполнению задач.

### Задание 1. Посчитайте, сколько книг вышло после 1 января 2000 года;

In [18]:
query1 = """
    SELECT COUNT(book_id) AS "Количество книг"
    FROM books
    WHERE publication_date > '2000-01-01'
"""
pd.io.sql.read_sql(query1, con = engine)

,Количество книг
0,819


Из 1000 книг после 1 января 2000 года вышло 819 книг.

### Задание 2. Для каждой книги посчитайте количество обзоров и среднюю оценку;

In [19]:
query2 = """
    SELECT
        ratings.book_id AS book_id,
        COUNT(DISTINCT reviews.review_id) AS "Количество обзоров",
        AVG(ratings.rating) AS "Средняя оценка"
    FROM reviews 
    LEFT JOIN ratings ON ratings.book_id = reviews.book_id
    GROUP BY ratings.book_id
    LIMIT 5
"""
pd.io.sql.read_sql(query2, con = engine)

,book_id,Количество обзоров,Средняя оценка
0,1,2,3.666667
1,2,1,2.500000
2,3,3,4.666667
3,4,2,4.500000
4,5,4,4.000000


Проверим правильность работы кода.

In [20]:
query2 = """
    SELECT 
        SUM("Количество обзоров") AS "Всего обзоров"
    FROM(
    SELECT
        ratings.book_id AS book_id,
        COUNT(DISTINCT reviews.review_id) AS "Количество обзоров",
        AVG(ratings.rating) AS "Средняя оценка"
    FROM reviews 
    LEFT JOIN ratings ON ratings.book_id = reviews.book_id
    GROUP BY ratings.book_id) as sub
    LIMIT 5
"""
pd.io.sql.read_sql(query2, con = engine)

,Всего обзоров
0,2793.0


Для каждой книги были посчитаны количество обзоров на нее и средняя оценка.

### Задание 3. Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;

In [21]:
query3 = """
    SELECT 
        publishers.publisher,
        COUNT(books.book_id) AS "Число выпущенных книг"
    FROM books 
    LEFT JOIN publishers ON publishers.publisher_id = books.publisher_id
    WHERE num_pages > 50
    GROUP BY publishers.publisher
    ORDER BY "Число выпущенных книг" DESC
    LIMIT 5
"""
pd.io.sql.read_sql(query3, con = engine)

,publisher,Число выпущенных книг
0,Penguin Books,42
1,Vintage,31
2,Grand Central Publishing,25
3,Penguin Classics,24
4,Bantam,19


Наибольшее число книг (42) выпустило издательство Penguin Books.

### Задание 4. Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;

In [22]:
query4 = """
    SELECT
        authors.author AS "Автор",
        COUNT(ratings.rating) AS "Количество оценок",
        AVG(ratings.rating) AS "Средняя оценка"
    FROM books 
    LEFT JOIN authors ON authors.author_id = books.author_id
    LEFT JOIN ratings ON ratings.book_id = books.book_id
    WHERE books.book_id IN (SELECT 
                                books.book_id  AS book_id
                            FROM books
                            LEFT JOIN ratings ON ratings.book_id = books.book_id
                            GROUP BY books.book_id
                            HAVING COUNT(ratings.rating) >= 50)
    GROUP BY authors.author
    ORDER BY "Средняя оценка" DESC
"""
pd.io.sql.read_sql(query4, con = engine)

,Автор,Количество оценок,Средняя оценка
0,J.K. Rowling/Mary GrandPré,310,4.287097
1,Markus Zusak/Cao Xuân Việt Khương,53,4.264151
2,J.R.R. Tolkien,162,4.246914
3,Louisa May Alcott,52,4.192308
4,Rick Riordan,62,4.080645
5,William Golding,71,3.901408
6,J.D. Salinger,86,3.825581
7,Paulo Coelho/Alan R. Clarke/Özdemir İnce,57,3.789474
8,William Shakespeare/Paul Werstine/Barbara A. M...,66,3.787879
9,Lois Lowry,56,3.750000


Автор с самой высокой средней оценкой книг - J.K. Rowling/Mary GrandPré.

### Задание 5. Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

In [24]:
query5 = """
    SELECT 
        AVG("Количество обзоров") AS "Среднее количество обзоров"
    FROM(
    
    SELECT 
        COUNT(review_id) AS "Количество обзоров"
    FROM reviews
    WHERE username IN (
                       SELECT 
                           username AS "Автор обзора"
                       FROM ratings
                       GROUP BY username
                       HAVING COUNT(rating_id) > 50) 
    GROUP BY username) AS FOO
    LIMIT 5
"""
pd.io.sql.read_sql(query5, con = engine)

,Среднее количество обзоров
0,24.333333


In [25]:
# ПОДЗАПРОС
query5 = """

    SELECT 
        username AS "Автор обзора",
        count(rating_id) as cnt_ratings
    FROM ratings
    GROUP BY username
    HAVING COUNT(rating_id) > 50
"""
pd.io.sql.read_sql(query5, con = engine)

,Автор обзора,cnt_ratings
0,sfitzgerald,55
1,jennifermiller,53
2,xdavis,51
3,paul88,56
4,martinadam,56
5,richard89,55


Среднее количество обзоров от пользователей, которые поставили больше 50 оценок - 24 штуки.